# Self Learning Tic-Tac-Toe Game by Nikola Kalev

#### Tik Tak Toe Game is relatively simple game. The game consists of total 26,830 unique game states. However, I'm trying to train NN (Neural Networ) how to win, block and and strategize effectively within the game.

#### As my objective is to iterate over a large number of games, I'm applying Q-Learning Technique which is part of the Reinforcment Learning

### Structure of the Notebook:
1. **Build a simple version of Tic Tak Toe Game - 3 X 3 (no visual imputs)**
    - add small helper functions

2. **Build Neural Net with Tensorflow and Keras**

3. **Buld Q-Learning Class**

4. **Train the agent to play against itself**

5. **Observe the results from the training**

6. **Build function so human can test and play against the trained agent**

7. **Next steps**

In [ ]:
import numpy as np
import pandas as pd
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

#### Build Simple Tic-Tak-Toe Game as class object in Python

In [ ]:
class TicTacToe:
    def __init__(self):
        self.board = [' ']*9
        self.player = 'O'

    def available_moves(self):
        return [i for i, x in enumerate(self.board) if x == ' ']

    def make_move(self, position, player):
        if self.board[position] == ' ':
            self.board[position] = player
            return True
        return False

    def check_win(self):
        win_conditions = [(0,1,2), (3,4,5), (6,7,8), (0,3,6), (1,4,7), (2,5,8), (0,4,8), (2,4,6)]
        for condition in win_conditions:
            if self.board[condition[0]] == self.board[condition[1]] == self.board[condition[2]] != ' ':
                return True
        if ' ' not in self.board:
            return True  # the game is over if the board is full
        return False

    def is_about_to_win(self, player):
        board = np.array(self.board).reshape((3, 3))

        # Check rows
        for row in board:
            if (row == player).sum() == 2 and (row == ' ').sum() == 1:
                return True

        # Check columns
        for col in board.T:  # .T transposes the board to treat columns as rows
            if (col == player).sum() == 2 and (col == ' ').sum() == 1:
                return True

        # Check diagonals
        diag1 = np.array([board[i, i] for i in range(3)])
        diag2 = np.array([board[i, 2 - i] for i in range(3)])
        if ((diag1 == player).sum() == 2 and (diag1 == ' ').sum() == 1) or \
        ((diag2 == player).sum() == 2 and (diag2 == ' ').sum() == 1):
            return True

        # If no two-in-a-row situation was found, return False
        return False

#### The following function transforms the board, allowing it to be read and understand by the Neural Network (NN).

In [ ]:
def one_hot_encode(board):
    transform = {'X': [1, 0, 0], 'O': [0, 1, 0], ' ': [0, 0, 1]}
    return np.array([transform[i] for i in board]).reshape(1, 9, 3)

#### Construct a Feed-forward Neural Network with a Dropout layer of 20%, using the Adam optimizer.

In [ ]:
def build_model():
    model = Sequential()
    model.add(Flatten(input_shape=(1, 9, 3)))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(9))

    optimizer = Adam(learning_rate=0.005)

    model.compile(loss='mse', optimizer=optimizer)
    return model


#### Build a Reinforcement Learning process using a Q-Learning Agent. The main objective is to enable the agent to learn by interacting with the environment.

In [ ]:
class QLearningAgent:
    def __init__(self, model, epsilon=0.1, discount_factor=0.7):
        self.model = model
        self.epsilon = epsilon
        self.discount_factor = discount_factor
        self.memory = []

    def choose_action(self, state, available_actions):
        if np.random.rand() < self.epsilon:
            return np.random.choice(available_actions)
        else:
            q_values = self.model.predict(state)[0]
            q_values = {action: q_values[action] for action in available_actions}
            return max(q_values, key=q_values.get)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((one_hot_encode(state).reshape(1, 9, 3), action, reward, one_hot_encode(next_state).reshape(1, 9, 3), done))

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))

        # Prepare batch data
        states_batch = np.array([np.array(experience[0]).reshape(1, 9, 3) for experience in minibatch])
        next_states_batch = np.array([np.array(experience[3]).reshape(1, 9, 3) for experience in minibatch])
        targets_batch = np.zeros((states_batch.shape[0], 9))  # Assuming 9 possible actions

        # Compute targets
        current_qs = self.model.predict(states_batch)
        future_qs = self.model.predict(next_states_batch)

        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            if done:
                target = reward
            else:
                target = reward + self.discount_factor * np.max(future_qs[i])
            current_qs[i][action] = target
            targets_batch[i] = current_qs[i]

        # Perform batch training
        self.model.fit(states_batch, targets_batch, epochs=1, verbose=0)

#### Train the agent - 1000 Games. Instead of having the agent play against random choices on the board, we set up an Agent vs Agent environment. This allows the agents to learn from each other moves and build strategies.

In [ ]:
def train_agent(agent1, agent2, games=1000, batch_size=32):
    for _ in range(games):
        game = TicTacToe()
        current_player = 'X'
        while not game.check_win():
            # The current agent takes its chosen action
            state = game.board.copy()
            available_moves = game.available_moves()
            if not available_moves: break  # the game is over
            
            if current_player == 'X':
                action = agent1.choose_action(one_hot_encode(state), available_moves)
            else:
                action = agent2.choose_action(one_hot_encode(state), available_moves)
                
            game.make_move(action, current_player)

            # Remember the state, action, reward, and next state
            next_state = game.board.copy()
            if game.check_win():
                if current_player == 'X':
                    reward = 1  # agent1 won
                    agent1.remember(state, action, reward, next_state, game.check_win())
                    agent2.remember(state, action, -reward, next_state, game.check_win())  # negative reward for losing
                else:
                    reward = 1  # agent2 won
                    agent2.remember(state, action, reward, next_state, game.check_win())
                    agent1.remember(state, action, -reward, next_state, game.check_win())  # negative reward for losing
            else:
                if game.is_about_to_win('X') and not game.is_about_to_win('O'):
                    reward = 2  # agent successfully blocked opponent
                    agent1.remember(state, action, reward, next_state, game.check_win())
                    agent2.remember(state, action, -reward, next_state, game.check_win())  # negative reward for not blocking
                elif game.is_about_to_win('O') and not game.is_about_to_win('X'):  # opponent is about to win, but agent didn't block
                    reward = -2.0  # heavily penalize the agent for not blocking
                    agent1.remember(state, action, reward, next_state, game.check_win())
                    agent2.remember(state, action, reward, next_state, game.check_win())
                else:
                    reward = -0.1  # encourage the agent to win quickly
                    agent1.remember(state, action, reward, next_state, game.check_win())
                    agent2.remember(state, action, reward, next_state, game.check_win())

            # Replay past experiences to learn
            agent1.replay(batch_size)
            agent2.replay(batch_size)
            
            # Swap players
            current_player = 'O' if current_player == 'X' else 'X'


#### Test the agents and store the results in a DataFrame.

In [ ]:

def test_agents(agent1, agent2, games=100):
    results = {'Agent1': {'Win': 0, 'Lose': 0, 'Draw': 0},
               'Agent2': {'Win': 0, 'Lose': 0, 'Draw': 0}}

    for _ in range(games):
        game = TicTacToe()
        current_player = 'X'
        while not game.check_win():
            # The current agent takes its chosen action
            state = game.board.copy()
            available_moves = game.available_moves()
            if not available_moves: break  # the game is over

            if current_player == 'X':
                action = agent1.choose_action(one_hot_encode(state), available_moves)
            else:
                action = agent2.choose_action(one_hot_encode(state), available_moves)

            game.make_move(action, current_player)

            if game.check_win():
                if current_player == 'X':
                    results['Agent1']['Win'] += 1
                    results['Agent2']['Lose'] += 1
                else:
                    results['Agent2']['Win'] += 1
                    results['Agent1']['Lose'] += 1
            elif ' ' not in game.board:  # the game is a draw
                results['Agent1']['Draw'] += 1
                results['Agent2']['Draw'] += 1

            # Swap players
            current_player = 'O' if current_player == 'X' else 'X'

    return pd.DataFrame(results)


In [ ]:
model1 = build_model()
agent1 = QLearningAgent(model1)

model2 = build_model()
agent2 = QLearningAgent(model2)

train_agent(agent1, agent2)

#### Print the results from the trained agents.

In [ ]:
results = test_agents(agent1, agent2, games = 100)

results

#### Build a Human Playgroud and Load the Trained agents:
Given that the Tic-Tac-Toe game favors the player who moves first, and considering the limited time I have for training the agent over a larger number of games, the code below gives the advantage to the agent (NN) by having it play first.

This could be changed once the agent is better trained.

In [ ]:
model1 = build_model()
agent1 = QLearningAgent(load_model("agent1_model_02062023.h5"))

model2 = build_model()
agent2 = QLearningAgent(load_model("agent2_model_02062023.h5"))

In [ ]:
def print_board(board):
    print("-------------")
    for i in range(0, 9, 3):
        print("|", board[i], "|", board[i+1], "|", board[i+2], "|")
        print("-------------")


## Agent play First:
def play_game(agent):
    game = TicTacToe()
    while not game.check_win():
        # The agent (player X) takes its chosen action
        state = game.board.copy()
        available_moves = game.available_moves()
        if available_moves:
            action = agent.choose_action(one_hot_encode(state), available_moves)
            game.make_move(action, 'X')
            if game.check_win():
                print('The agent won.')
                break

        # Player O takes a move based on user input
        print('Current board:')
        print_board(game.board)
        available_moves = game.available_moves()
        if not available_moves:
            print('It is a draw.')
            break
        print('Available moves:', available_moves)
        move = int(input('Your move (0-8): '))
        while move not in available_moves:
            print('Invalid move.')
            move = int(input('Your move (0-8): '))
        game.make_move(move, 'O')
        if game.check_win():
            print('You won!')
            break

## Human play First:
# def play_game(agent):
#     game = TicTacToe()
#     while not game.check_win():
#         # Player X takes a move based on user input
#         print('Current board:')
#         print_board(game.board)
#         available_moves = game.available_moves()
#         print('Available moves:', available_moves)
#         move = int(input('Your move (0-8): '))
#         while move not in available_moves:
#             print('Invalid move.')
#             move = int(input('Your move (0-8): '))
#         game.make_move(move, 'X')
#         if game.check_win():
#             print('You won!')
#             break

#         # The agent (player O) takes its chosen action
#         state = game.board.copy()
#         available_moves = game.available_moves()
#         if available_moves:
#             action = agent.choose_action(one_hot_encode(state), available_moves)
#             game.make_move(action, 'O')
#             if game.check_win():
#                 print('The agent won.')
#                 break


##### Play the game:

In [ ]:
play_game(agent1)

#### Next Steps:
1. Train the agent over a much larger sample of games.

2. Make modifications to the Neural Network, such as:
    - Change the hyperparameters
    - Making it deeper and adding more Dropout layers
    - Changing the optimizers
    

3. Further exploration why there is no draw games in the results